In [1]:
import cvxpy as cp
import numpy as np
import time
start_time = time.time()


# First, we write the function for creating box
# This is the code to make a box around a vector v = [v1 v2 ... vn]^T with parameters r = [r1 r2 ... rn]^T
def make_box(v, r):
    n = len(v)
    zeros = [0]*(n+1)
    mat = []
    # Adding the constraints for positive values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)
    # Adding the constraints for negative values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)   
    # print('mat zeros = ', mat) # [[0.1,0,0], [0.1,0,0], [0.1,0,0], [0.1,0,0]]
    
    for i in range(n):
        mat[i][i+1] = 1 # [[0.1,1,0], [0.1,0,1], [0.1,0,0], [0.1,0,0]]
    # print('mat pos = ', mat)
    j = 1
    for i in range(n,2*n):
        mat[i][j] = -1 # # [[0.1,1,0], [0.1,0,1], [0.1,-1,0], [0.1,0,-1]]
        j += 1
    # print('mat for constraints = ', mat)
    Box = Polyhedron(ieqs=mat, backend='ppl', base_ring=QQ)
    return Box

# We write a function to compute minimum distance between two polyhedra, and return the points
def min_dist_between_polyhedra(P1, P2):
    # At first, we extract the linear constraints from both polyhedra written as Ax + b >= 0
    Ab1 = list(P1.Hrepresentation())
    Ab2 = list(P2.Hrepresentation())
    b1, A1 = ([a[0] for a in Ab1], [a[1:] for a in Ab1])
    b2, A2 = ([a[0] for a in Ab2], [a[1:] for a in Ab2])
    b1, A1, b2, A2 = np.array(b1), np.array(A1), np.array(b2), np.array(A2)
    # print('b1 = ', b1)
    # print('A1 = ', A1)
    # print('b2 = ', b2)
    # print('A2 = ', A2)

    # The dimension of the polyhedra are the number of columns in Matrix A
    n = len(A1[0])
    print('n = ', n)

    # We define two vectors of same dimensions
    X1 = cp.Variable(n)
    X2 = cp.Variable(n)

    # Next, we write the constraints for the two polyhedra P1 and P2 such that vector X1 in P1 and X2 in P2
    constraints = [A1@X1 >= -b1, A2@X2 >=-b2]

    # Finally, we write the objective function which is to minimize the infinity norm of any two points in P1 and P2
    objective = cp.Minimize(cp.norm_inf(X1-X2))

    # Problem formulation
    prob = cp.Problem(objective, constraints)

    # Solve the problem
    prob.solve()

    # Finally, we have the minimum distance and the two vectors,
    val, x1, x2 = prob.value, X1.value, X2.value
    # print('Minimum distance = ', val)
    # print('Vector in P1 = ', x1)
    # print('Vector in P2', x2)
    return val, x1, x2

# Next, we write a function to compute the maximum distance of a vector and a polyhedron
def max_dist_vector_to_polyhedron(v, P):
    X = P.vertices_list()
    dist_list = [(np.linalg.norm(v-x), x) for x in X]
    d, vx = max(dist_list)
    return d, vx

# Select the file to take input for algorithm 1
v = input("Enter the example number to run algorithm 1: ")
exp_num = v
# Evaluate the inputs
if (int(v)==1):
    input_file = open(r"inputs/ex1_dummy.txt")
elif (int(v)==2):
    input_file = open(r"inputs/ex2_balance_system.txt")
elif (int(v)==3):
    input_file = open(r"inputs/ex3_3deg_quadcopter.txt")
elif (int(v)==4):
    input_file = open(r"inputs/ex4_6deg_quadcopter.txt")

# Execute the input file to get A, B, K, X0, U0
exec(input_file.read())
input_file.close()

# Create the box
v = [0]*n
r = [ep]*n
Box = make_box(v,r)
# print('Box = ', Box.vertices())

# Opening a file to write the outputs
exp_string = "outputs/alg3/ex" + exp_num + ".txt"
f = open(exp_string, "a")
f.write('\nNew Experiment:\n')


# Running algorithm 3
Xc = X0
Uc = U0
l, u = T0[0], T0[1]
iter = 50
for i in range(iter):
    try:
        print('Iteration = ', i, file=f)
        Al = A*l
        Bl = B*l
        AlXc = Al*Xc
        BlUc = Bl*Uc
        X1 = AlXc+BlUc
        print("X1 = ", X1, file=f)
        Au = A*u
        Bu = B*u
        X2 = Au*Xc + Bu*Uc
        print("X2 = ", X2, file=f)
        X3 = Polyhedron(vertices=(X1.vertices_list()+X2.vertices_list()), backend='ppl', base_ring=QQ) # Convex hull of X1, X2
        print("X3 = ", X3)
        Xc = X3 + C*Xc + D*Uc + E
        print('Number_of_Cons(Xc) = ', len(list(Xc.Hrepresentation())), file=f)
        size = c/l
        v, r = [0]*n, [size]*n
        Xh = Xc + make_box(v, r)
        print('Number_of_Cons(Xh) = ', len(list(Xh.Hrepresentation())), file=f)
        Uc = K*Xh
        print('Number_of_Cons(Uc) = ', len(list(Uc.Hrepresentation())), file=f)
        # CVX tool steps
        l, x1, w1 = min_dist_between_polyhedra(Xh, W)
        u, x2 = max_dist_vector_to_polyhedron(w1, Xh)
        # Xw = Xc & W
        # if (Xw.vertices_list()):
        #     print(Xw)
        #     print("Unsafe state reached!!!\n", file=f)
        #     break
    except ValueError as ve:
        print('ValueError = ', ve, file=f)
        break
    except ZeroDivisionError as ze:
        print("ZeroDivisionError = ", ze, file=f)
        
#         print('i = ', i, file=f)
#         AXc = A*Xc
#         # print("AXc = ", AXc, file=f)
#         BUc = B*Uc
#         # print("BUc = ", BUc, file=f)
#         Xc = AXc + BUc    
#         print('Xc = ', Xc, file=f)
#         # print('Box = ', Box)
#         Xp = Xc + Box
#         print('Xp = ', Xp, file=f)
#         Uc = K*Xp
#         print('Uc = ', Uc, file=f)
#     except ValueError as ve:
#         print('ValueError = ', ve, file=f)

# Print the output State
print("\nThe constraints of output Xc = ", list(Xc.Hrepresentation()), file=f)

# Save the runtime
end_time = time.time()
total_time_in_ms = (end_time-start_time)*10**3
f.write('\nTotal runtime = '+str(round(total_time_in_ms, 4))+' ms for '+str(iter)+' iterations\n')

# Close the file
f.close()

# Saving the plot
try:
    fig = Xc.plot()
    fig_string = "outputs/alg1/ex" + exp_num + ".png"
    fig.save(fig_string)
except:
    pass

# Plotting in notebook
Xc.vertices_list()

NameError: name 'input_file' is not defined